<a href="https://colab.research.google.com/github/ehzawad/adversarialsomething/blob/main/adversarialsomething.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ehzawad/adversarial-lesions.git

Cloning into 'adversarial-lesions'...
remote: Enumerating objects: 439, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 439 (delta 136), reused 163 (delta 67), pack-reused 200
Receiving objects: 100% (439/439), 69.40 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (245/245), done.


In [2]:
!cp -R /content/drive/MyDrive/isic2016 /content/dataset

In [4]:
!mv /content/adversarial-lesions/helpers.py /content/adversarial-lesions/isic2016_scripts

In [10]:
%cd /content/adversarial-lesions/

/content/adversarial-lesions


In [11]:
!pwd

/content/adversarial-lesions


In [12]:
!python /content/adversarial-lesions/isic2016_scripts/data_preprocess_isic2016.py

Training data...
900it [00:52, 17.18it/s]
(900, 256, 256, 3) (900, 2)
Done!
Test data...
379it [00:19, 19.63it/s]
(379, 256, 256, 3) (379, 2)
Done!


In [13]:
!python /content/adversarial-lesions/isic2016_scripts/data_preprocess_gan.py

900it [01:22, 10.95it/s]
Done!


In [14]:
!ls -al /content/dataset

total 20
drwxr-xr-x 5 root root 4096 Aug 24 22:34 .
drwxr-xr-x 1 root root 4096 Aug 24 22:06 ..
drwx------ 4 root root 4096 Aug 24 22:18 isic2016
drwxr-xr-x 4 root root 4096 Aug 24 22:34 isic2016gan
drwxr-xr-x 2 root root 4096 Aug 24 22:33 isic2016numpy


In [15]:
!ls -al /content/dataset/isic2016gan

total 32
drwxr-xr-x 4 root root  4096 Aug 24 22:34 .
drwxr-xr-x 5 root root  4096 Aug 24 22:34 ..
drwxr-xr-x 2 root root 20480 Aug 24 22:35 trainA
drwxr-xr-x 2 root root  4096 Aug 24 22:35 trainB


In [16]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-n8emm1qr
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-n8emm1qr
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=b2e80bc98641a15f5a4ef3020dea0fc1dda21bcc1933d6595e46d5fd59b309bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-awc8mpxu/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [23]:
import os
import tensorflow as tf
import scipy
from glob import glob
import numpy as np

print(tf.test.gpu_device_name())

base_path = os.path.abspath("../")
dataset_path = os.path.join(base_path, "dataset/isic2016gan")
model_path = os.path.join(base_path, "models")
print(os.listdir(dataset_path))


class DataLoader():
    def __init__(self, dataset_name, img_res=(256, 256)): #128
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('{}/dataset/%s/%s/*'.format(base_path) % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        # rescale to [-1, 1]
        imgs = np.array(imgs)/127.5 - 1.
        
        # rescale to [0, 1]
        #imgs = imgs.astype('float32')
        #imgs = np.array(imgs)
        #imgs = imgs / 255.
        
        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('{}/dataset/%s/%sA/*'.format(base_path) % (self.dataset_name, data_type))
        path_B = glob('{}/dataset/%s/%sB/*'.format(base_path) % (self.dataset_name, data_type))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            # rescale to [-1, 1]
            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            # rescale to [0, 1]
            
            #imgs_A = imgs_A.astype('float32')
            #imgs_A = np.array(imgs_A)
            #imgs_A = imgs_A / 255.
            
            #imgs_B = imgs_B.astype('float32')
            #imgs_B = np.array(imgs_B)
            #imgs_B = imgs_B / 255.
            
            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        
        # rescale to [-1, 1]
        img = img/127.5 - 1.
        
        # rescale to [0, 1]
        #img = img.astype('float32')
        #img = img / 255.
        
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float) # np.float


['trainB', 'trainA']


In [24]:
#Sanity check
path = glob('{}/dataset/%s/%s/*'.format(base_path) % ("isic2016gan", "trainB"))
path[:3]

['/content/dataset/isic2016gan/trainB/554.jpeg',
 '/content/dataset/isic2016gan/trainB/194.jpeg',
 '/content/dataset/isic2016gan/trainB/105.jpeg']